<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=54dc346ec20cb3cde9020e212755b55d1def45e7c215b2b1dfdedcde4cfcc76d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-04 14:26:56
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.44 C
-------------------
Today PnL: -89.33 K (-0.61%)
Current PnL: -20.10 L (-13.17%)
CY Booked + Current PnL: -7.45 L (-4.88%)
-------------------
Total profit:  2.15 L
Total loss:  -22.25 L
-------------------
Total Booked + Current PnL: 19.53 L (15.54%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.09 L (60.29%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 8.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.66,65.0,X-LC,5.58,230740.0,20374.0,8399.0,-0.38,9.69,3.64,13.68,37.0,2.43,1.61,27.81,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,68.0,M-LC,10.19,172597.0,31757.0,12738.0,0.61,22.55,7.38,31.59,66.0,2.49,1.20,55.01,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.84,54.0,M-SC,1.72,87796.0,-12981.0,13073.0,-1.25,-12.88,14.89,0.09,245.0,-0.99,0.61,15.34,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.22,64.0,X-MC,5.26,150997.0,14864.0,15719.0,1.64,10.92,10.41,22.47,91.0,0.95,1.05,25.31,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.86,39.0,X-MC,2.51,88020.0,-2658.0,17569.0,1.05,-2.93,19.96,16.44,101.0,-0.15,0.61,16.65,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,735.00,-7.44,57.0,X-MC,3.05,250769.0,2701.0,105699.0,2.73,1.09,42.15,43.70,102.0,0.03,1.75,15.69,XY24,BTT,FMCG
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.06,6.69,XY24,NTT,AUTO
33,ICICIGI,2252.93,-12.22,64.0,X-MC,5.26,150997.0,14864.0,15719.0,1.64,10.92,10.41,22.47,91.0,0.95,1.05,25.31,X40,ATH,INSURANCE
59,REPCOHOME,880.00,-52.31,75.0,H-SC,6.74,274361.0,-12144.0,269477.0,1.32,-4.24,98.22,89.82,134.0,-0.05,1.91,45.16,XY24,NTT,FINANCE
42,JCHAC,2282.00,19921.74,63.0,M-SC,1.50,97888.0,-29917.0,29905.0,1.29,-23.41,30.55,-0.01,232.0,-1.00,0.68,10.84,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TITAGARH,1548.00,-6.26,46.0,H-SC,3.56,200298.0,-41879.0,152647.0,-3.14,-17.29,76.21,45.74,158.0,-0.27,1.40,30.27,XY24,NTT,ENGINEERING
49,MASFIN,398.61,-17.80,54.0,H-SC,8.78,93510.0,-4470.0,26071.0,-2.65,-4.56,27.88,22.05,152.0,-0.17,0.65,35.92,XR,ATH,FINANCE
16,CAMS,4762.00,-6.49,44.0,X-SC,0.79,250734.0,-1122.0,63561.0,-2.52,-0.45,25.35,24.79,122.0,-0.02,1.75,22.13,X40N,NTT,MISC
4,ALKYLAMINE,4546.37,-14.76,39.0,H-SC,4.73,79361.0,-21602.0,116137.0,-2.37,-21.40,146.34,93.63,148.0,-0.19,0.55,18.21,SR,ATH,CHEMICALS
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204325.0,1155.0,40967.0,-2.03,0.57,20.05,20.74,55.0,0.03,1.42,7.47,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204325.0,1155.0,40967.0,-2.03,0.57,20.05,20.74,55.0,0.03,1.42,7.47,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.96,45.0,M-LC,10.66,205037.0,-39.0,112606.0,-1.93,-0.02,54.92,54.89,52.0,-0.00,1.43,24.07,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.84,54.0,M-SC,1.72,87796.0,-12981.0,13073.0,-1.25,-12.88,14.89,0.09,245.0,-0.99,0.61,15.34,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.62,220959.0,-48708.0,85047.0,-1.46,-18.06,38.49,13.47,138.0,-0.57,1.54,12.35,XY24,NTT,PAINTS
17,CERA,9475.0,-21.24,57.0,H-SC,1.93,143497.0,-32406.0,74432.0,-0.70,-18.42,51.87,23.89,149.0,-0.44,1.00,23.93,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.38,43.0,H-MC,6.04,102220.0,-29615.0,71840.0,-0.99,-22.46,70.28,32.03,98.0,-0.41,0.71,14.50,OX40N,NTT,IT
42,JCHAC,2282.0,19921.74,63.0,M-SC,1.50,97888.0,-29917.0,29905.0,1.29,-23.41,30.55,-0.01,232.0,-1.00,0.68,10.84,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,68.0,M-LC,10.19,172597.0,31757.0,12738.0,0.61,22.55,7.38,31.59,66.0,2.49,1.2,55.01,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,58.0,M-MC,9.70,235570.0,10608.0,157055.0,-0.85,4.72,66.67,74.53,192.0,0.07,1.64,38.49,XY24,BTT,STEEL
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204325.0,1155.0,40967.0,-2.03,0.57,20.05,20.74,55.0,0.03,1.42,7.47,XY25,NTT,FINANCE
46,KPIGREEN,731.05,13.24,73.0,H-SC,6.54,134959.0,9662.0,49260.0,-0.60,7.71,36.50,47.03,141.0,0.20,0.94,68.98,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,68.0,M-LC,10.19,172597.0,31757.0,12738.0,0.61,22.55,7.38,31.59,66.0,2.49,1.20,55.01,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.63,126298.0,11500.0,82548.0,-0.40,10.02,65.36,81.92,223.0,0.14,0.88,49.52,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,13.24,73.0,H-SC,6.54,134959.0,9662.0,49260.0,-0.60,7.71,36.50,47.03,141.0,0.20,0.94,68.98,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.63,126298.0,11500.0,82548.0,-0.40,10.02,65.36,81.92,223.0,0.14,0.88,49.52,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,55.0,H-SC,6.07,130491.0,7155.0,124475.0,-0.19,5.80,95.39,106.72,119.0,0.06,0.91,30.54,AR,ATH,MISC
31,HINDZINC,730.22,25.96,45.0,M-LC,10.66,205037.0,-39.0,112606.0,-1.93,-0.02,54.92,54.89,52.0,-0.00,1.43,24.07,X5K,ATH,METALS
86,WIPRO,420.00,-16.06,39.0,M-LC,5.41,147430.0,-3515.0,112976.0,-1.13,-2.33,76.63,72.51,53.0,-0.03,1.03,3.55,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.06,6.69,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.72,27.0,X-SC,6.33,88561.0,-40109.0,85408.0,-0.33,-31.17,96.44,35.20,219.0,-0.47,0.62,1.65,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-22.32,32.0,X-MC,4.83,195143.0,-5204.0,35711.0,-0.29,-2.60,18.30,15.23,178.0,-0.15,1.36,28.82,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-12.79,34.0,X-LC,7.02,244710.0,-12353.0,47498.0,-0.52,-4.81,19.41,13.67,24.0,-0.26,1.71,14.62,XY25,NTT,FMCG
51,PAGEIND,51769.93,-29.00,35.0,X-MC,7.09,80240.0,-3270.0,23302.0,-1.57,-3.92,29.04,23.98,82.0,-0.14,0.56,1.68,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.92,48.0,X-LC,1.17,200148.0,10.0,20875.0,-1.12,0.01,10.43,10.44,4.0,0.00,1.40,5.57,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.04,47.0,H-LC,1.23,155880.0,-30215.0,77691.0,-0.32,-16.24,49.84,25.51,15.0,-0.39,1.09,14.87,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,39.0,X-MC,1.28,200970.0,150.0,67647.0,0.05,0.07,33.66,33.76,80.0,0.00,1.40,5.28,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.46,45.0,X-MC,2.90,312354.0,-15637.0,122162.0,-0.66,-4.77,39.11,32.48,92.0,-0.13,2.18,6.11,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.81,75.0,X-LC,4.25,249624.0,15480.0,33150.0,0.26,6.61,13.28,20.77,86.0,0.47,1.74,13.99,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.31,37.0,X-SC,37.75,50587.0,-14419.0,165349.0,-1.88,-22.18,326.86,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.72,27.0,X-SC,6.33,88561.0,-40109.0,85408.0,-0.33,-31.17,96.44,35.20,219.0,-0.47,0.62,1.65,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.00,35.0,X-MC,7.09,80240.0,-3270.0,23302.0,-1.57,-3.92,29.04,23.98,82.0,-0.14,0.56,1.68,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.30,42.0,X-MC,7.36,218500.0,-44865.0,154174.0,-0.70,-17.04,70.56,41.51,84.0,-0.29,1.52,2.54,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,44.0,X-MC,2.45,183460.0,-54491.0,207145.0,-1.92,-22.90,112.91,64.15,174.0,-0.26,1.28,2.91,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.31,37.0,X-SC,37.75,50587.0,-14419.0,165349.0,-1.88,-22.18,326.86,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.00,35.0,X-MC,7.09,80240.0,-3270.0,23302.0,-1.57,-3.92,29.04,23.98,82.0,-0.14,0.56,1.68,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.27,42.0,X-SC,4.46,81413.0,-63747.0,139672.0,-0.32,-43.92,171.56,52.31,136.0,-0.46,0.57,8.98,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.86,39.0,X-MC,2.51,88020.0,-2658.0,17569.0,1.05,-2.93,19.96,16.44,101.0,-0.15,0.61,16.65,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.72,27.0,X-SC,6.33,88561.0,-40109.0,85408.0,-0.33,-31.17,96.44,35.20,219.0,-0.47,0.62,1.65,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.65,45.0,X-LC,10.08,280910.0,-65046.0,131747.0,-0.94,-18.80,46.90,19.28,1.0,-0.49,1.96,3.85,X40,ATH,IT
39,INFY,2275.00,-18.88,53.0,X-LC,5.37,316412.0,3524.0,168173.0,0.22,1.13,53.15,54.87,3.0,0.02,2.21,9.50,X40,BTT,IT
41,ITC,452.00,-37.92,48.0,X-LC,1.17,200148.0,10.0,20875.0,-1.12,0.01,10.43,10.44,4.0,0.00,1.40,5.57,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,7.21,299860.0,-15982.0,128640.0,-1.00,-5.06,42.90,35.67,5.0,-0.12,2.09,8.09,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,52.0,H-LC,5.22,251544.0,-10075.0,129294.0,-0.29,-3.85,51.40,45.57,7.0,-0.08,1.75,7.03,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,28.08,78394.0,-35155.0,75203.0,-1.04,-30.96,95.93,35.27,268.0,-0.47,0.55,91.87,XR,NTT,HOTELS
7,ASIANTILES,137.00,7472.22,70.0,L-SC,9.82,83062.0,-10748.0,87365.0,1.07,-11.46,105.18,81.67,269.0,-0.12,0.58,60.85,XR,NTT,CERAMICS
49,MASFIN,398.61,-17.80,54.0,H-SC,8.78,93510.0,-4470.0,26071.0,-2.65,-4.56,27.88,22.05,152.0,-0.17,0.65,35.92,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.63,126298.0,11500.0,82548.0,-0.40,10.02,65.36,81.92,223.0,0.14,0.88,49.52,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,55.0,H-SC,6.07,130491.0,7155.0,124475.0,-0.19,5.80,95.39,106.72,119.0,0.06,0.91,30.54,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,79.06,71.0,H-SC,1.73,155438.0,-27337.0,138402.0,-2.01,-14.96,89.04,60.77,125.0,-0.20,1.08,39.22,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-52.31,75.0,H-SC,6.74,274361.0,-12144.0,269477.0,1.32,-4.24,98.22,89.82,134.0,-0.05,1.91,45.16,XY24,NTT,FINANCE
46,KPIGREEN,731.05,13.24,73.0,H-SC,6.54,134959.0,9662.0,49260.0,-0.60,7.71,36.50,47.03,141.0,0.20,0.94,68.98,MH,ATH,POWER
62,SAMMAANCAP,326.00,-197.73,71.0,M-SC,32.23,168282.0,19062.0,125118.0,-1.55,12.77,74.35,96.62,208.0,0.15,1.17,82.12,XY25,NTT,FINANCE
68,SONACOMS,806.63,-32.06,67.0,M-SC,7.25,84630.0,-16529.0,56533.0,0.49,-16.34,66.80,39.54,202.0,-0.29,0.59,19.34,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.67
1,25,44.10
2,50,75.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.04
LC    32.95
MC    23.00
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.05
X40      16.14
X40N     12.44
XY25     11.88
XR        9.92
AR        7.83
OX40N     6.55
X200      1.76
X5K       1.43
SR        1.05
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.36
X-LC    22.30
X-MC    18.70
M-SC    12.70
X-SC     5.51
M-LC     5.08
H-LC     4.60
H-MC     2.34
M-MC     1.64
L-SC     1.47
L-LC     0.97
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.23,-3.63,37.13
IT,12.38,-18.32,81.71
FINANCE,12.18,-8.16,57.18
MISC,7.75,-16.92,75.91
ELECTRICAL,5.79,-8.80,47.37
PAINTS,5.44,-18.30,35.93
BANKS,4.06,-16.09,83.63
INSURANCE,3.86,-0.02,37.31
AUTO,2.79,-44.13,104.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3266997.0,22
XR,1234249.0,13
AR,1164008.0,9
X40,842309.0,12
X40N,684984.0,9
OX40N,545317.0,9
XY25,467970.0,8
SR,269832.0,2
X5K,112606.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3336299.0,24
M-SC,1223736.0,15
X-MC,1192681.0,14
X-LC,1081201.0,13
X-SC,559548.0,6
M-LC,279287.0,4
H-LC,278315.0,3
H-MC,251731.0,2
L-SC,251722.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1170670.0      6
           AR         833767.0      5
           XR         770743.0      7
M-SC       XY24       702474.0      6
X-MC       XY24       570703.0      4
X-LC       X40        468922.0      5
X-SC       X40N       308791.0      4
X-LC       XY24       308117.0      2
X-MC       X40        287979.0      6
H-SC       SR         269832.0      2
           OX40N      242027.0      3
X-LC       X40N       222068.0      3
H-LC       AR         206985.0      2
H-MC       XY24       179891.0      1
X-MC       XY25       179874.0      2
X-SC       XY24       165349.0      1
L-SC       XR         162568.0      2
M-MC       XY24       157055.0      1
X-MC       X40N       154125.0      2
M-SC       OX40N      142296.0      4
           XR         130592.0      2
           XY25       125118.0      1
           AR         123256.0      2
M-LC       XR         112976.0      1
           X5K        112606.0      1
L-SC       OX40N       89154.0      1
X-SC       X40         85408.0      1
X-LC       XY25        82094.0      3
H-MC       OX40N       71840.0      1
H-LC       X200        71330.0      1
L-MC       XR          57370.0      1
H-SC       MH          49260.0      1
M-LC       XY25        40967.0      1
L-LC       XY25        39917.0      1
M-LC       XY24        12738.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
